In [12]:
import csv
import os
import pandas as pd
import keras.backend as K
import numpy as np
import csv
import nltk
import re
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import *
from keras.preprocessing.text import Tokenizer,one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.utils import *
from re import split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from nltk import word_tokenize, pos_tag
from nltk.tokenize import RegexpTokenizer
from stanfordcorenlp import StanfordCoreNLP
from collections import Counter
from listofimg import listOfimg
from time import time
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
import random
from random import choice

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/xuesiyuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
path_traindata = '/home/xuesiyuan/pythonworkspace/DailyDialogue/dailydialog/dialogues_text.txt'
path_trainlabel = '/home/xuesiyuan/pythonworkspace/DailyDialogue/dailydialog/dialogues_act.txt'

In [14]:
def load_data():
    utterance = []
    act = []
    f1 = open(path_traindata,'r')
    f2 = open(path_trainlabel,'r')
    for item in f1.readlines():
        i = item.strip('\n')
        s = i.split(' __eou__')
        for item in s:
            if item != '':
                utterance.append(item)

    for item in f2.readlines():
        i = item.strip('\n')
        split = i.split()
        for i in split:
            act.append(i)
    return utterance,act

def get_data(act_tag,utterance):
    act_dic = dict() #lenth 43
    cc = list(zip(act_tag,utterance))
    for i in range(len(act_tag)):
        if act_tag[i] not in act_dic.keys():
            act_dic[act_tag[i]] = [cc[i][1]]
        if act_tag[i] in act_dic.keys():
            act_dic[act_tag[i]].append(cc[i][1])

    actdict_list = []
    label_list = []
    for key,values in act_dic.items():
        actdict_list.append(values)
        label_list.append(key)

    #padding
    flatten = sum(actdict_list,[])
    tokenizer = Tokenizer(filters='\n')
    tokenizer.fit_on_texts(flatten)
    flatten = tokenizer.texts_to_sequences(flatten)
    padding = pad_sequences(flatten)   
    
    data0 = []
    count = 0
    for i in actdict_list:
        l = len(i)
        data0.append(padding[count:count+l].tolist())
        count += l
    return (data0,label_list)


def split(data): #先在整个数据集上随机抽取出8/2用作训练和测试
    random.seed(1)
    train = []
    test = []
    for i in data:
        a = int(len(i)*0.6)
        random.shuffle(i)
        train.append(i[:a])
        test.append(i[a:])
    return (train,test)


def load_data_label(train,label_list):
    X = []
    y = []
    for i,j in zip(train,label_list):
#         print (i,j)
        for ii in i:
            X.append(ii)
            y.append(j)
    X = np.array(X)
    y = np.array(y)
    return (X,y)
   
    
def process_word(utterance):
    ##vocab
    tokenizer = Tokenizer(filters='\n')
    tokenizer.fit_on_texts(utterance)
    word_index = tokenizer.word_index
    vocab_size = len(tokenizer.word_index)
#     input_utt_sequence = tokenizer.texts_to_sequences(anchor_utt)
#     input_anchor_utt = pad_sequences(input_utt_sequence, maxlen = input_max_length)
    print('--------------build embeding index---------------')
    embeddings_index = dict()
    f = open('/home/xuesiyuan/pythonworkspace/swda/glove.6B.50d.txt')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))    
    embedding_matrix = np.zeros((vocab_size+1, 50))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    
    return  vocab_size,embedding_matrix

In [24]:
def conv_1D_bn(x, filters, kernel_size, stride=1, dilation_rate=1, padding='same', activation='relu'):
    x = Conv1D(filters=filters, kernel_size=kernel_size, strides=stride,dilation_rate=dilation_rate, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation is not None:
        activation_x = Activation(activation)(x)
    return activation_x
    
def inceptionRes1D(input):
    x0 = Activation('relu')(input)
    x00 = conv_1D_bn(x0, 128, 1)
    x1 = conv_1D_bn(x0, 32, 1)#1gram
    x2 = conv_1D_bn(x0, 32, 2)#2gram
    x3 = conv_1D_bn(x0, 32, 1, dilation_rate=2)
    x4 = conv_1D_bn(x0, 32, 2, dilation_rate=2)
    x11 = Concatenate()([x1,x2])
    x11 = conv_1D_bn(x11, 128, 1)
    x_con = Add()([x00, x11])
    return(x_con)



def attention_3d_block(inputs): #input 4
    a = Dense(1, activation='tanh')(inputs)
    a = Flatten()(a)
    a = RepeatVector(128)(a)
    a = Dense(input_max_length, activation='softmax')(a)
    output_attention_mul = dot([a, inputs],axes = [2,1], name='attention_mul')
    return output_attention_mul
  
    
    
def train_model():
    
    utt_input = Input(shape = (input_max_length,))
    input_em = Embedding(input_dim = vocab_size+1, 
                     output_dim = 50, 
                     weights = [embedding_matrix], 
                     input_length = input_max_length, 
                     trainable = False)(utt_input)
    cnn_encode = inceptionRes1D(input_em)
    dropout1 = Dropout(0.5)(cnn_encode)
    att_out = attention_3d_block(dropout1)
    gru_encode = GRU(hidden_dim, name='lstm',return_sequences=True,recurrent_dropout=0.5)(att_out)
    flatten = Flatten()(gru_encode)
    utt_output = Dense(128, activation='sigmoid')(flatten)
    utterance_model = Model(inputs = utt_input, outputs = utt_output,name = 'utterance_model')
    utterance_model.summary()    
  

    #######################################################################################################

    anchor_input = Input(shape = (input_max_length,),name='anchor_input') 
    positive_input = Input(shape = (input_max_length,),name='pos_input')
    negative_input = Input(shape = (input_max_length,),name='neg_input')
    
    anchor_out = utterance_model(anchor_input)
    positive_out = utterance_model(positive_input)
    negative_out = utterance_model(negative_input)
    
    inputs = [anchor_input,positive_input,negative_input]
    outputs = [anchor_out,positive_out,negative_out]
    
    triplet_model = Model(inputs, outputs)
    triplet_model.add_loss(K.mean(triplet_loss(outputs)))
#     triplet_model.summary()
    
    return utterance_model,triplet_model

In [16]:
def triplet_loss(inputs, dist='sqeuclidean', margin='maxplus'):
    head_input,positive_input,negative_input = inputs
    positive_distance = K.square(head_input - positive_input)#平方和
    negative_distance = K.square(head_input - negative_input)
    
    if dist == 'euclidean':
        positive_distance = K.sqrt(K.sum(positive_distance, axis=-1, keepdims=True)) 
        negative_distance = K.sqrt(K.sum(negative_distance, axis=-1, keepdims=True))
    elif dist == 'sqeuclidean':
        positive_distance = K.sum(positive_distance, axis=-1, keepdims=True)
        negative_distance = K.sum(negative_distance, axis=-1, keepdims=True)
        
    loss = positive_distance - negative_distance
    
    if margin == 'maxplus':
        loss = K.maximum(0.0, 1 + loss) #margin = 1
    elif margin == 'softplus':
        loss = K.log(1 + K.exp(loss))
        
    return K.mean(loss)

In [17]:
def generator(data0,batch_size):  
    count = 0
    while True:
        training = []
        for _ in range(batch_size):
            anchor_pos_pair = random.sample(data0,2)
            anchor_pos = random.sample(anchor_pos_pair[0],2)
            [neg] = random.sample(anchor_pos_pair[1],1)
            anchor_pos.append(neg)
            training.append(anchor_pos)
        trans = np.array(training).transpose(1,0,2)
        count +=1
        yield ([trans[0],trans[1],trans[2]],None)

In [18]:
input_max_length= 278
batch_size = 128
hidden_dim = 128
num_label = 4

In [19]:
utterance,act = load_data()
data0,label_list = get_data(act,utterance)
train,test = split(data0)
vocab_size,embedding_matrix = process_word(utterance)

--------------build embeding index---------------
Loaded 400000 word vectors.


In [20]:
for i in data0:
    print(len(i))

29429
9725
17296
46533


In [21]:
print(utterance[2])
print(act[2])
print(data0[1][1])
print(label_list[1])

So Dick , how about getting some coffee for tonight ?
3
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 117, 25, 22, 3, 76, 1]
4


In [22]:
utterance_path ='./modelsave/1219_DailyDialogue_utterance.h5'
triplet_path = './modelsave/1219_DailyDialogue_triplet.h5'
result_path = './result_model/1219_DailyDialogue_model.h5'
log_dir = './1219_DailyDialogue_embeddingmodel_CNN/ATT/GRU'
log_dir_model = './result_logs'

In [ ]:
%%time
class SaveSubModel(Callback):
    def on_train_begin(self, logs={}):
        self.losses = 10
        self.val_loss = 10
        
    def on_epoch_end(self, epoch, logs={}):
        if self.losses > logs.get('loss'):
            print ('updata loss from %.4f to %.4f, save utterance_model' % (self.losses, logs.get('loss')))
            self.losses = logs.get('loss')
            utterance_model.save(utterance_path )
            
        if self.val_loss > logs.get('val_loss'):
            print ('updata loss from %.4f to %.4f, save triplet_model' % (self.val_loss, logs.get('val_loss')))
            self.val_loss = logs.get('val_loss')
            triplet_model.save(triplet_path)


save_model = SaveSubModel()
optimizer = optimizers.Adam(lr=0.001, decay=0.0001, clipnorm=1.0)#
early_stopping = EarlyStopping(monitor='val_loss', patience=100)
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0,write_graph=True, write_images=False)


utterance_model,triplet_model = train_model()
triplet_model.compile(loss = None, optimizer = optimizer)
history = triplet_model.fit_generator(generator = generator(train,batch_size),
                                      nb_epoch=1000,
                                      steps_per_epoch = 100,
                                      validation_data = generator(test,batch_size),
                                      validation_steps= 100,
                                      callbacks = [save_model,early_stopping,tensorboard])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 278)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 278, 50)       1180500     input_2[0][0]                    
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 278, 50)       0           embedding_2[0][0]                
____________________________________________________________________________________________________
conv1d_8 (Conv1D)                (None, 278, 32)       1632        activation_8[0][0]               
___________________________________________________________________________________________

/home/xuesiyuan/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:25: UserWarning: Output "utterance_model" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "utterance_model" during training.
/home/xuesiyuan/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:31: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/xuesiyuan/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(steps_per_epoch=100, validation_data=<generator..., validation_steps=100, cal

Epoch 1/1000
 99/100 [============================>.] - ETA: 3s - loss: 1.0090updata loss from 10.0000 to 1.0090, save utterance_model
updata loss from 10.0000 to 1.0000, save triplet_model
100/100 [==============================] - 462s - loss: 1.0090 - val_loss: 1.0000
Epoch 2/1000
100/100 [==============================] - 459s - loss: 1.0000 - val_loss: 1.0000
Epoch 3/1000
100/100 [==============================] - 458s - loss: 1.0000 - val_loss: 1.0000
Epoch 4/1000
100/100 [==============================] - 458s - loss: 1.0000 - val_loss: 1.0000
Epoch 5/1000
100/100 [==============================] - 459s - loss: 1.0000 - val_loss: 1.0000
Epoch 6/1000
100/100 [==============================] - 459s - loss: 1.0000 - val_loss: 1.0000
Epoch 7/1000
100/100 [==============================] - 458s - loss: 1.0000 - val_loss: 1.0000
Epoch 8/1000
100/100 [==============================] - 459s - loss: 1.0000 - val_loss: 1.0000
Epoch 9/1000
100/100 [==============================] - 459s - 